In [9]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import pandas as pd

In [17]:
url = f'https://meeyland.com/mua-ban-nha-dat/page-{page}'

In [26]:
file_name = 'data.csv'
headers = ['Price', 'Location', 'Area', 'Street', 'Facade', 'Direction', 'Object', 'Type']
f = open(file_name, "w")   
f.write('\t'.join(headers))
f.write('\n')

1

In [7]:
def get_info(html_text):
    info = []
    price = html_text.find('div', {'class':'article-price'}).string.strip('\n         ')
    location = html_text.find('div', {'class':'article-info-location'}).string.strip('\n         ')
    details = results[0].find_all('div', {'class':'col-6'})
    details = [detail.text.strip('\n\n               ').replace('---', '') for detail in details]
    info.append(price)
    info.append(location)
    info.extend(details)
    return info
page = 1

while True:
    try:
        print(url)
        html_text = requests.get(url).text
        tree = BeautifulSoup(html_text, 'html.parser')
#         print(html_text)
    except Exception as e:
        print(f'Error: {e}')
        break
    results = tree.find_all('div', {'class':'col-sm-6 col-md-4'})
    info = get_info(results[0])
    info = '\t'.join(info)
    f.write(info+'\n')
    if page > 92001:
        break
    page += 1


https://meeyland.com/mua-ban-nha-dat/page-1
390 Triệu - 1.96 Triệu/m²	Huyện Củ Chi, Thành phố Hồ Chí Minh	198.7 m²	Ngõ 1 ô tô	12.9m	Tây	Nhà môi giới	Đất - Đất nền - Nhà như đất


In [27]:
f.close()